In [ ]:
import pickle
from google.colab import drive
import os
import pprint
import pandas as pd
from tensorflow.keras.utils import plot_model
from keras.models import load_model,Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
from keras.utils import np_utils

In [ ]:

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.27.96.50:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 7369304745454012255),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -3219567743174558984),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, -1561601111619124330),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, -7338362977615657662),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, -6876150849496566102),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1599602500828088086),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 233774835397318511),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1217812115579935014),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 2

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.27.96.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.27.96.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with open("/content/drive/MyDrive/FT_300/sarcasm/sarcasm_params.pickle",'rb') as file:
  ft_file = pickle.load(file)

with open("/content/drive/MyDrive/GLOVE_100/sarcasm/sarcasm_params.pickle",'rb') as file:
  gl100_file = pickle.load(file)

with open("/content/drive/MyDrive/GLOVE_200/sarcasm/sarcasm_params.pickle",'rb') as file:
  gl200_file = pickle.load(file)

In [ ]:

# df = pd.read_csv('/content/drive/MyDrive/sentiment data/tr_sen_fin.csv')
df = pd.read_csv('/content/drive/MyDrive/datasets/train_sarc_final_with_folds.csv')

In [ ]:
df_tr = df[df['kfold']!=1].reset_index(drop=True)
df_val = df[df['kfold']==1].reset_index(drop=True)

In [ ]:
tr_text = df_tr['text'].values
val_text = df_val['text'].values

In [ ]:

y_val = np_utils.to_categorical(df_val.lbl_num.values)
y_tr = np_utils.to_categorical(df_tr.lbl_num.values)

y_val = tf.convert_to_tensor(y_val,np.float32)
y_tr = tf.convert_to_tensor(y_tr,np.float32)



In [ ]:
# names = ['GLOVE_200D_BiLSTM.h5','GLOVE_100D_CONV1D_BiGRU.h5','FT_300D_CONV1D.h5']
# 'GLOVE_100D_CONV_1D_BiGRU.h5'
# names = ['FT_300/sarcasm/sarcasm_SEP_CONV1D.h5','FT_300/sarcasm/sarcasm_BiGRU.h5','GLOVE_100/sarcasm/sarcasm_CONV1D.h5','GLOVE_200/sarcasm/sarcasm_CONV1D_BiLSTM.h5']
names = ['FT_300/sarcasm/sarcasm_BiLSTM.h5','FT_300/sarcasm/sarcasm_CONV1D.h5','GLOVE_200/sarcasm/sarcasm_BiRNN.h5','GLOVE_200/sarcasm/sarcasm_SEP_CONV1D_BiGRU.h5']
# members = load_all_models(names)

In [ ]:
def load_all_models(names):
  with tf.device('/job:localhost'):
      # for name in names:
      # # define filename for this ensemble
      #   filename = '/content/drive/MyDrive/sentiment models/'+name
      # # load model from file
      # models = [load_model('/content/drive/MyDrive/sarcasm models/'+name) for name in names]
    models = [load_model('/content/drive/MyDrive/'+name) for name in names]
      # add to list of members
        # all_models.append(model)
        # print('>loaded %s' % name)
  return models
  


In [ ]:
# from keras.utils import plot_model
def define_stacked_model(names,members):
	# update all layers in all models to not be trainable
  for i in range(len(members)):
    model = members[i]
    # model.input._name = names[i].replace('.h5','')+'_'+model.input.name
    for layer in model.layers:
      # make not trainable
      layer.trainable = False
      # rename to avoid 'unique layer name' issue
      layer._name = names[i].replace('.h5','')+'_' + 'ensemble'+'_'+str(i+1) + '_' + layer.name
    # define multi-headed input
  ensemble_visible = [model.input for model in members]
  # concatenate merge output from each model
  ensemble_outputs = [model.output for model in members]
  merge = concatenate(ensemble_outputs)
  hidden = Dense(4, activation='relu')(merge)
  output = Dense(2, activation='softmax')(hidden)
  model = Model(inputs=ensemble_visible, outputs=output)
  # plot graph of ensemble
  plot_model(model, show_shapes=True, to_file='model_graph.png')
  # compile
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [ ]:
with strategy.scope():
  members = load_all_models(names)
  stacked = define_stacked_model(members)

In [ ]:
plot_model(stacked, show_shapes=True, to_file='model_graph.png')

In [ ]:
def folds(x):
  df_tr = df[df['kfold']!=x].reset_index(drop=True)
  df_val = df[df['kfold']==x].reset_index(drop=True)
  tr_text = df_tr['text'].values
  val_text = df_val['text'].values  
  x_tr_gl_100 = tf.convert_to_tensor(pad_sequences(gl100_file['Tokenizer'].texts_to_sequences(tr_text),gl100_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_tr_gl_200 = tf.convert_to_tensor(pad_sequences(gl200_file['Tokenizer'].texts_to_sequences(tr_text),gl200_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_tr_ft_300 = tf.convert_to_tensor(pad_sequences(ft_file['Tokenizer'].texts_to_sequences(tr_text),ft_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)


  # val_text = df_val['text'].values
  x_val_gl_100 = tf.convert_to_tensor(pad_sequences(gl100_file['Tokenizer'].texts_to_sequences(val_text),gl100_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_val_gl_200 = tf.convert_to_tensor(pad_sequences(gl200_file['Tokenizer'].texts_to_sequences(val_text),gl200_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_val_ft_300 = tf.convert_to_tensor(pad_sequences(ft_file['Tokenizer'].texts_to_sequences(val_text),ft_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)

  y_val = np_utils.to_categorical(df_val.lbl_num.values)
  y_tr = np_utils.to_categorical(df_tr.lbl_num.values)

  y_val = tf.convert_to_tensor(y_val,np.float32)
  y_tr = tf.convert_to_tensor(y_tr,np.float32)

  return {'x_tr_gl_100':x_tr_gl_100,'x_tr_gl_200':x_tr_gl_200,'x_tr_ft_300':x_tr_ft_300,'y_tr':y_tr,'x_val_gl_100':x_val_gl_100,'x_val_gl_200':x_val_gl_200,'x_val_ft_300':x_val_ft_300,'y_val':y_val}

In [ ]:
initial_learning_rate = 0.01
epochs = 100
decay = initial_learning_rate / epochs
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

In [ ]:

# stacked.fit(x = [x_tr_gl_200,x_tr_gl_100,x_tr_ft_300],y = y_tr,validation_data=([x_val_gl_200,x_val_gl_100,x_val_ft_300],y_val),epochs=100,callbacks=[EarlyStopping()])
def train_stacked():
  stacked1 = ['FT_300/sarcasm/sarcasm_SEP_CONV1D.h5','FT_300/sarcasm/sarcasm_BiGRU.h5','GLOVE_100/sarcasm/sarcasm_CONV1D.h5','GLOVE_200/sarcasm/sarcasm_CONV1D_BiLSTM.h5']
  stacked2 = ['FT_300/sarcasm/sarcasm_BiLSTM.h5','FT_300/sarcasm/sarcasm_CONV1D.h5','GLOVE_200/sarcasm/sarcasm_BiRNN.h5','GLOVE_200/sarcasm/sarcasm_SEP_CONV1D_BiGRU.h5']
  stacked3 = ['FT_300/sarcasm/sarcasm_CONV1D_BiLSTM.h5','FT_300/sarcasm/sarcasm_BiGRU.h5','GLOVE_100/sarcasm/sarcasm_CONV1D_BiRNN.h5','GLOVE_100/sarcasm/sarcasm_BiRNN.h5']
  stacked4 = ['FT_300/sarcasm/sarcasm_BiLSTM.h5','FT_300/sarcasm/sarcasm_CONV1D.h5','GLOVE_100/sarcasm/sarcasm_BiRNN.h5','GLOVE_100/sarcasm/sarcasm_CONV1D_BiGRU.h5']
  stacked5 = ['FT_300/sarcasm/sarcasm_CONV1D.h5','GLOVE_100/sarcasm/sarcasm_BiGRU.h5','GLOVE_200/sarcasm/sarcasm_CONV1D_BiLSTM.h5','GLOVE_200/sarcasm/sarcasm_BiLSTM.h5']
  stacked6 = ['FT_300/sarcasm/sarcasm_CONV1D.h5','FT_300/sarcasm/sarcasm_BiLSTM.h5','FT_300/sarcasm/sarcasm_CONV1D_BiLSTM.h5','FT_300/sarcasm/sarcasm_SEP_CONV1D.h5','FT_300/sarcasm/sarcasm_BiGRU.h5']
  stacked7 = ['GLOVE_100/sarcasm/sarcasm_CONV1D.h5','GLOVE_100/sarcasm/sarcasm_CONV1D_BiRNN.h5','GLOVE_100/sarcasm/sarcasm_BiRNN.h5','GLOVE_100/sarcasm/sarcasm_CONV1D_BiGRU.h5','GLOVE_100/sarcasm/sarcasm_BiGRU.h5']
  stacked8 = ['GLOVE_200/sarcasm/sarcasm_CONV1D_BiLSTM.h5','GLOVE_200/sarcasm/sarcasm_BiRNN.h5','GLOVE_200/sarcasm/sarcasm_SEP_CONV1D_BiGRU.h5','GLOVE_200/sarcasm/sarcasm_BiLSTM.h5']

  model_config = {}
  # model_config['STACKED_1'] = (stacked1,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_100','x_tr_gl_200'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_200']) #300 300 100 200
  # model_config['STACKED_2'] = (stacked2,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_200','x_tr_gl_200'],['x_val_ft_300','x_val_ft_300','x_val_gl_200','x_val_gl_200']) #300 300 200 200
  # model_config['STACKED_3'] = (stacked3,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_100','x_tr_gl_100'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_100']) #300 300 100 100
  # model_config['STACKED_4'] = (stacked4,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_100','x_tr_gl_100'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_100']) #300 300 100 100
  # model_config['STACKED_5'] = (stacked5,['x_tr_ft_300','x_tr_gl_100','x_tr_gl_200','x_tr_gl_200'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_100']) #300 100 200 200
  model_config['STACKED_6'] = (stacked6,['x_tr_ft_300','x_tr_ft_300','x_tr_ft_300','x_tr_ft_300','x_tr_ft_300'],['x_val_ft_300','x_val_ft_300','x_val_ft_300','x_val_ft_300','x_val_ft_300']) #300 300 300 300 300
  model_config['STACKED_7'] = (stacked7,['x_tr_gl_100','x_tr_gl_100','x_tr_gl_100','x_tr_gl_100','x_tr_gl_100'],['x_val_gl_100','x_val_gl_100','x_val_gl_100','x_val_gl_100','x_val_gl_100']) #100 100 100 100 100
  model_config['STACKED_8'] = (stacked8,['x_tr_gl_200','x_tr_gl_200','x_tr_gl_200','x_tr_gl_200'],['x_val_gl_200','x_val_gl_200','x_val_gl_200','x_val_gl_200']) #200 200 200 200 

  for k,v in model_config.items():
    with strategy.scope():
      members = load_all_models(v[0])
      stacked = define_stacked_model(v[0],members)
    
    plot_model(stacked, show_shapes=True, to_file='/content/drive/MyDrive/STACKED/sarcasm/sarcasm_'+k+'.png')
    print("----------------------::  TRAINING STARTED :- "+k+" ::----------------------\n")
    for i in range(5):
      print("-----------:: FOLD :- "+str(i+1)+' ::-----------')
      data = folds(i)
      x_t = [data[x] for x in v[1]]
      y_t = data['y_tr']
      x_v = [data[x] for x in v[2]]
      y_v = data['y_val']
      stacked.fit(x = x_t,y = y_t,validation_data=(x_v,y_v),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=2),ModelCheckpoint(filepath='/content/drive/MyDrive/STACKED/sarcasm/sarcasm_'+k+'.h5', monitor='val_loss', save_best_only=True),LearningRateScheduler(lr_time_based_decay,verbose=1)])
    print("----------------------::  TRAINING COMPLETED :- "+k+" ::----------------------\n")

In [ ]:
train_stacked()

----------------------::  TRAINING STARTED :- STACKED_6 ::----------------------

-----------:: FOLD :- 1 ::-----------
Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
7350/7350 [==============================] - 276s 31ms/step - loss: 0.2579 - accuracy: 0.9044 - val_loss: 0.2826 - val_accuracy: 0.8845
Epoch 2/100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0009999000574917021.
7350/7350 [==============================] - 224s 30ms/step - loss: 0.2295 - accuracy: 0.9058 - val_loss: 0.2835 - val_accuracy: 0.8847
Epoch 3/100

Epoch 00003: LearningRateScheduler reducing learning rate to 0.000999700106714659.
7350/7350 [==============================] - 220s 30ms/step - loss: 0.2298 - accuracy: 0.9060 - val_loss: 0.2837 - val_accuracy: 0.8849
-----------:: FOLD :- 2 ::-----------
Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0009997001616284251.
7350/7350 [==============================] - 22

In [ ]:
ft_file

In [ ]:
# from keras.preprocessing.text import Tokenizer
ft_x_val = ft_file['Tokenizer'].texts_to_sequences(val_text)
gl100_x_val = gl100_file['Tokenizer'].texts_to_sequences(val_text)
gl200_x_val = gl100_file['Tokenizer'].texts_to_sequences(val_text)


In [ ]:

#x_val_padded = pad_sequences(x_val, object_file[1], padding='post',truncating = 'post')

In [ ]:
dic = {}
for i in range(len(members)):
  model = members[i]
  model.input._name = names[i].replace('.h5','')+'_'+model.input.name
  #   layer._name = names[i].replace('.h5','')+layer.name
  #   mod.append(layer.name)
  # dic[names[i]] = mod

In [ ]:
ensemble_visible = [model.input for model in members]

In [ ]:
ensemble_visible[0].

In [ ]:
from collections import Counter
l = []
for layers in dic.values():
  l.extend(layers)

c = Counter(l)
print('\n\n\n',c,'\n\n\n')

In [ ]:
stacked = define_stacked_model(members)

In [ ]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))

In [ ]:
dataset = np.hstack((in_seq1, in_seq2))

In [ ]:
os.listdir()

In [ ]:
# import pickle
# # gl_200 = pickle.load(open('/content/drive/MyDrive/sarcasm models/gl_200_prep_params.pkl','rb'))
# # gl_100 = pickle.load(open('/content/drive/MyDrive/sarcasm models/gl_100_prep_params.pkl','rb'))
# # ft_300 = pickle.load(open('/content/drive/MyDrive/sarcasm models/ft_300_prep_params.pkl','rb'))
# gl_200 = pickle.load(open('/content/drive/MyDrive/sentiment models/gl_200_prep_params.pkl','rb'))
# gl_100 = pickle.load(open('/content/drive/MyDrive/sentiment models/gl_100_prep_params.pkl','rb'))
# ft_300 = pickle.load(open('/content/drive/MyDrive/sentiment models/ft_300_prep_params.pkl','rb'))


ft_file

In [ ]:
import gensim

model_100 = gensim.models.Word2Vec(sentences=df['text'].values,size = 100,window=5,workers=4,min_count=1)
model_200 = gensim.models.Word2Vec(sentences=df['text'].values,size = 200,window=5,workers=4,min_count=1)
model_300 = gensim.models.Word2Vec(sentences=df['text'].values,size = 300,window=5,workers=4,min_count=1)

In [ ]:
model_100.wv.save_word2vec_format('/content/drive/MyDrive/sarcasm models/sarc_w2v_100D.txt',binary=False)
model_200.wv.save_word2vec_format('/content/drive/MyDrive/sarcasm models/sarc_w2v_200D.txt',binary=False)
model_300.wv.save_word2vec_format('/content/drive/MyDrive/sarcasm models/sarc_w2v_300D.txt',binary=False)

In [ ]:
len(list(model_100.wv.vocab))

In [ ]:
!wget https://drive.google.com/u/0/open?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [ ]:
# SARCASM STACKING

# Epoch 1/3
# 1838/1838 [==============================] - 334s 177ms/step - loss: 0.4566 - accuracy: 0.7794
# Epoch 2/3
# 1838/1838 [==============================] - 325s 177ms/step - loss: 0.2568 - accuracy: 0.9022
# Epoch 3/3
# 1838/1838 [==============================] - 324s 176ms/step - loss: 0.2437 - accuracy: 0.9029
# <tensorflow.python.keras.callbacks.History at 0x7f262d19dc90>
# [18]


In [ ]:
# !gunzip /content/drive/MyDrive/w2v_emb/GoogleNews-vectors-negative300.bin.gz

In [ ]:
with tf.device('/job:localhost'):

In [ ]:
# Sentiment DATA

# Epoch 1/100
# 17250/17250 [==============================] - 371s 21ms/step - loss: 0.5395 - accuracy: 0.8170 - val_loss: 0.4415 - val_accuracy: 0.8477
# Epoch 2/100
# 17250/17250 [==============================] - 363s 21ms/step - loss: 0.4656 - accuracy: 0.8381 - val_loss: 0.4354 - val_accuracy: 0.8481
# Epoch 3/100
# 17250/17250 [==============================] - 362s 21ms/step - loss: 0.4606 - accuracy: 0.8386 - val_loss: 0.4315 - val_accuracy: 0.8486
# Epoch 4/100
# 17250/17250 [==============================] - 364s 21ms/step - loss: 0.4576 - accuracy: 0.8390 - val_loss: 0.4313 - val_accuracy: 0.8481
# Epoch 5/100
# 17250/17250 [==============================] - 362s 21ms/step - loss: 0.4579 - accuracy: 0.8382 - val_loss: 0.4288 - val_accuracy: 0.8488
# Epoch 6/100
# 17250/17250 [==============================] - 361s 21ms/step - loss: 0.4566 - accuracy: 0.8387 - val_loss: 0.4281 - val_accuracy: 0.8486
# Epoch 7/100
# 17250/17250 [==============================] - 361s 21ms/step - loss: 0.4556 - accuracy: 0.8389 - val_loss: 0.4289 - val_accuracy: 0.8485